<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="../assets/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 4. Interlinked Plots</h2></div>

Using hvPlot allows you to generate a number of different types of plot
quickly from a standard API by building [HoloViews](https://holoviews.org) objects, as discussed in the previous
notebook. These objects are rendered with Bokeh which offers a number of
standard ways to interact with your plot, such as panning and zooming
tools.

Many other modes of interactivity are possible when building an
exploratory visualization (such as a dashboard) and these forms of
interactivity cannot be achieved using hvPlot alone.

In this notebook, we will drop down to the HoloViews level of
representation to build a visualization directly that consists of linked plots that
update when you interactivity select a particular earthquake with the
mouse. The goal is to show how more sophisticated forms of interactivity can be built when needed, in a way that's fully compatible with all the examples shown in earlier sections.

First let us load our initial imports:

In [ ]:
import numpy as np
import holoviews as hv
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas  # noqa
from datashader.utils import lnglat_to_meters
from holoviews.element import tiles

And clean the data before filtering (for magnitude `>7`) and projecting to to Web Mercator as before:

In [ ]:
df = dd.read_parquet('../data/earthquakes.parq').repartition(npartitions=4)
cleaned_df = df.copy()
cleaned_df['mag'] = df.mag.where(df.mag > 0)
cleaned_reindexed_df = cleaned_df.set_index(cleaned_df.time)
cleaned_reindexed_df = cleaned_reindexed_df.persist()

most_severe = cleaned_reindexed_df[cleaned_reindexed_df.mag >= 7].compute()
x, y = lnglat_to_meters(most_severe.longitude, most_severe.latitude)
most_severe_projected = most_severe.join([pd.DataFrame({'easting': x}), pd.DataFrame({'northing': y})])

Towards the end of the previous notebook we generated a scatter plot of earthquakes
across the earth that had a magnitude `>7` that was projected using 
datashader and overlaid on top of a map tile source:

In [ ]:
high_mag_quakes = most_severe_projected.hvplot.points(x='easting', y='northing', c='mag', 
                                                      title='Earthquakes with magnitude >= 7')
esri = tiles.ESRI().redim(x='easting', y='northing')
esri * high_mag_quakes

And saw how this object is a HoloViews `Points` object:

In [ ]:
print(high_mag_quakes)

This object is an example of a HoloViews *Element* which is an object that can display itself. These elements are *thin* wrappers around your data and the raw input data is always available on the `.data` attribute. For instance, we can look at the `head` of the `most_severe_projected` `DataFrame` as follows:

In [ ]:
high_mag_quakes.data.head()

In [ ]:
# Linked selections work when trying to link different subviews/subplots of the same dataset.

# Note a dataset, doesn't have to be the same thing in memory. Up to the user to make sure semantics make sense.

In [ ]:
ls = hv.link_selections.instance()

mag_hist = most_severe_projected.hvplot(
    y='mag', kind='hist', responsive=True, min_height=200
)

depth_hist = most_severe_projected.hvplot(
    y='depth', kind='hist', responsive=True, min_height=200
)

In [ ]:
geo = most_severe_projected.hvplot(
    'easting', 'northing', color='mag', kind='points',
    xaxis=None, yaxis=None, responsive=True, min_height=500,
)

geo

In [ ]:
ls(depth_hist)+ ls(mag_hist)

In [ ]:
(hv.element.tiles.ESRI() * ls(geo)) + ls(depth_hist)

In [ ]:
# Show dataframe. 

# 1. Plot as GUI to select data.
# 2. Chaining the filtered data into new plots

In [ ]:
ls.filter(most_severe_projected).dframe()

### The `help` system (introduce when .opts needed)

You can learn more about the `.opts` method and the HoloViews options
system in the [corresponding user
guide](http://holoviews.org/user_guide/Applying_Customizations.html). To
easily learn about the available options from inside a notebook, you can
use `hv.help` and inspect the 'Style Options'.

In [ ]:
# Commented as there is a lot of help output!
# hv.help(hv.Scatter) 

At this point, we can have some insight to the sort of HoloViews object
hvPlot is building behind the scenes for our earthquake example:

In [ ]:
esri * hv.Points(most_severe_projected, ['easting', 'northing'], 'mag').opts(color='mag', size=8, aspect='equal')

#### Exercise

Try using `hv.help` to inspect the options available for different element types such as the `Points` element used above. Copy the line above into the cell below and pick a `Points` option that makes sense to you and try using it in the `.opts` method.

<details><summary>Hint</summary><br>

If you can't decide on an option to pick, a good choice is `marker`. For instance, try:

 * `marker='+'` 
 * `marker='d'`.
 
 HoloViews uses [matplotlib's conventions](https://matplotlib.org/3.1.0/api/markers_api.html) for specifying the various marker types. Try finding out which ones are support by Bokeh.

</details>

## Conclusion

When exploring data it can be convenient to use the `.plot` API to quickly visualize a particular dataset. By calling `.plot` to generate different plots over the course of a session, it is possible to gradually build up a mental model of how a particular dataset is structured. While this works well for simple datasets, it can be more efficient to build a linked visualization with support for direct user interaction as a tool for more rapidly gaining insight.

In the workflow presented here, building such custom interaction is relatively quick and easy and does not involve throwing away prior code used to generate simpler plots. In the spirit of 'short cuts not dead ends', we can use the HoloViews output of `hvplot` that we used in our initial exploration to build rich visualizations with custom interaction to explore our data at a deeper level. 

These interactive visualizations not only allow for custom interactions beyond the scope of `hvplot` alone, but they can display visual annotations not offered by the `.plot` API. In particular, we can overlay our data on top of tile sources, generate interactive textual annotations, draw shapes such a circles, mark horizontal and vertical marker lines and much more. Using HoloViews you can build visualizations that allow you to directly interact with your data in a useful and intuitive manner.

In this notebook, the earthquakes plotted were either filtered early on by magnitude (`>=7`) or dynamically to analyse only the earthquakes within a small geographic distance. This allowed us to use Bokeh directly without any special handing and without having to worry about the performance issues that would be occur if we were to try to render the whole dataset at once.

In the next section we will see how such large datasets can be visualized directly using Datashader.